<a href="https://colab.research.google.com/github/KhalidSwaid/CloudCourse/blob/main/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install requests beautifulsoup4
!pip install firebase

In [2]:
from firebase import firebase

In [3]:
fbConn = firebase.FirebaseApplication('https://lab6-87d9f-default-rtdb.firebaseio.com/',None)

In [4]:
import requests
from bs4 import BeautifulSoup

def fetch_page(url):
  response = requests.get(url)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup
  else:
    return None


In [5]:
import re
def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    #print(format_json(index))
    return index

In [6]:
def remove_stop_words(index):
  stop_words = {'a', 'an', 'the', 'and', 'or','in', 'on', 'at'}
  for stop_word in stop_words:
      if stop_word in index:
            del index[stop_word]
  return index

In [7]:
from nltk.stem import PorterStemmer
def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count
    return stemmed_index

In [8]:
def search(query, index):
    query_words = re.findall(r'\w+', query.lower())
    results = {}
    for word in query_words:
        if word in index:
            results[word] = index[word]
    return results

In [9]:
def search_engine(url, query):
    soup = fetch_page(url)
    if soup is None:
        return None
    index = index_words(soup)
    index = remove_stop_words(index)
    index = apply_stemming(index)
    results = search(query, index)
    return results

In [10]:
from urllib.parse import urlparse, urljoin
from collections import deque

# Function to extract links from a webpage
def extract_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for link in soup.find_all('a', href=True):
        links.append(link['href'])
    return links

# Function to check if links are from the same domain
def check_same_domain(base_url, links):
    base_domain = urlparse(base_url).netloc
    same_domain_links = []
    for link in links:
        parsed_link = urlparse(link)
        if parsed_link.netloc == base_domain or parsed_link.netloc == '':
            same_domain_links.append(link)
    return same_domain_links

def is_same_domain(base_url, url):
    base_domain = urlparse(base_url).netloc
    link_domain = urlparse(url).netloc
    return base_domain == link_domain

In [11]:
def fetch_page_title(url):
    soup = fetch_page(url)
    if soup:
        title_tag = soup.find('title')
        if title_tag:
            return title_tag.get_text()
    return None

In [12]:
import hashlib

def save_and_hash_url(url_to_hash):
  # Hash the URL using MD5 hashing algorithm
  hashed_url = hashlib.md5(url_to_hash.encode()).hexdigest()

  title = fetch_page_title(url_to_hash)
  data_to_upload = {
    'url': url_to_hash,
    'title': title
  }
  result = fbConn.put_async('/pages', hashed_url,data=data_to_upload)
  return hashed_url

In [13]:

import urllib.parse

def search_words(url,query_words):
  hashed_url = save_and_hash_url(url)
  #print(hashed_url)
  for query in query_words:
    result = search_engine(url, query)
    if result == {}:
      continue
    #print(result," ---- ",result[query])
    #data_to_upload ={
    #    'link': url,
    #    'count': result[query]
    #}
    data_to_upload = {
        'count': result[query]
    }
    result = fbConn.put_async('/index/' + query,hashed_url,data=data_to_upload)

In [18]:
start_url = "https://www.ibm.com/cloud"
queries = ["ai","public","private","cloud","server","database"]
#queries = ["infrastructure","Development","production","holdings"]

###BFS

visited = set()
# Initialize a queue with first url
queue = deque([start_url])

#get the base domain to start from
#base_domain = urlparse(start_url).netloc

# Loop until the queue is empty
while queue:
  #get next url to search
  url = queue.popleft()
  # Skip if the URL has been visited before
  if url in visited:
      continue

  # mark visited
  visited.add(url)

  print("Visiting:", url)
  #Get all the links in this page
  links = extract_links(url)
  print("Links Extracted: ",len(links))

  #searching words and saving them to firebase

  search_words(url,queries)
  # Ppocess link
  for link in links:
    absolute_link = urljoin(url, link)

    if is_same_domain(start_url, absolute_link) and absolute_link not in visited:
      #add link to queue
      queue.append(absolute_link)

Visiting: https://www.ibm.com/cloud
Links Extracted:  5
Visiting: https://www.ibm.com/products/security-and-compliance-center
Links Extracted:  0
Visiting: https://www.ibm.com/products/satellite
Links Extracted:  2
Visiting: https://www.ibm.com/products/vmware
Links Extracted:  2
Visiting: https://www.ibm.com/cloud/financial-services
Links Extracted:  0
Visiting: https://www.ibm.com/cloud/solutions
Links Extracted:  0
Visiting: https://www.ibm.com/account/reg/signup?formid=urx-49524
Links Extracted:  2
Visiting: https://www.ibm.com/partnerplus
Links Extracted:  2
Visiting: https://www.ibm.com/downloads/cas/QE7BZ7A5
Links Extracted:  0


KeyboardInterrupt: 